# 3.4.0_Dialogue

In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
import pandas as pd
import numpy as np

import sqlite3

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [11]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'c'
project = 'Building Chatbots in Python'
data = make_dir_line(modality, project)

raw = data('raw')

## 3.4.2 Form filling

No he construido la funcion send_message

In [12]:
# # Define the INIT state
# INIT = 0

# # Define the CHOOSE_COFFEE state
# CHOOSE_COFFEE = 1

# # Define the ORDERED state
# ORDERED = 2

# # Define the policy rules
# policy = {
#     (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
#     (INIT, "none"): (INIT, "I'm sorry - I'm not sure how to help you"),
#     (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
#     (CHOOSE_COFFEE, "none"): (CHOOSE_COFFEE, "I'm sorry - would you like Colombian or Kenyan?"),
# }

# # Create the list of messages
# messages = [
#     "I'd like to become a professional dancer",
#     "well then I'd like to order some coffee",
#     "my favourite animal is a zebra",
#     "kenyan"
# ]

# # Call send_message() for each message
# state = INIT
# for message in messages:    
#     state = send_message(policy, state, message)

Respuesta:
    USER : I'd like to become a professional dancer
    BOT : I'm sorry - I'm not sure how to help you
    USER : well then I'd like to order some coffee
    BOT : ok, Colombian or Kenyan?
    USER : my favourite animal is a zebra
    BOT : I'm sorry - would you like Colombian or Kenyan?
    USER : kenyan
    BOT : perfect, the beans are on their way!

## 3.4.3 Asking contextual questions

In [ ]:
# # Define the states
# INIT=0 
# CHOOSE_COFFEE=1
# ORDERED=2

# # Define the policy rules dictionary
# policy_rules = {
#     (INIT, "ask_explanation"): (INIT, "I'm a bot to help you order coffee beans"),
#     (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
#     (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
#     (CHOOSE_COFFEE, "ask_explanation"): (CHOOSE_COFFEE, "We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost $6. The Brazilian beans make a nutty coffee and cost $5.")    
# }

# # Define send_messages()
# def send_messages(messages):
#     state = INIT
#     for msg in messages:
#         state = send_message(state, msg)

# # Send the messages
# send_messages([
#     "what can you do for me?",
#     "well then I'd like to order some coffee",
#     "what do you mean by that?",
#     "kenyan"
# ])

Respuesta:

    USER : what can you do for me?
    BOT : I'm a bot to help you order coffee beans
    USER : well then I'd like to order some coffee
    BOT : ok, Colombian or Kenyan?
    USER : what do you mean by that?
    BOT : We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost $6. The Brazilian beans make a nutty coffee and cost $5.
    USER : kenyan
    BOT : perfect, the beans are on their way!

## 3.4.4 Dealing with rejection

In [ ]:
# Define find_hotels()
def find_hotels(params):
    # Create the base query
    query = 'SELECT * FROM hotels'
    # Add filter clauses for each of the parameters
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        query += " WHERE " + " and ".join(filters)
    # Create the tuple of values
    t = tuple(params.values())
    
    # Open connection to DB
    conn = sqlite3.connect(raw / 'hotels.db')
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query, t)
    # Return the results
    return c.fetchall()

In [ ]:
# # Define respond()
# def respond(message, params, prev_suggestions, excluded):
#     # Interpret the message
#     parse_data = interpret(message)
#     # Extract the intent
#     intent = parse_data["intent"]["name"]
#     # Extract the entities
#     entities = parse_data["entities"]
#     # Add the suggestion to the excluded list if intent is "deny"
#     if intent == "deny":
#         excluded.extend(prev_suggestions)
#     # Fill the dictionary with entities
#     for ent in entities:
#         params[ent["entity"]] = str(ent["value"])
#     # Find matching hotels
#     results = [
#         r 
#         for r in find_hotels(params, excluded) 
#         if r[0] not in excluded
#     ]
#     # Extract the suggestions
#     names = [r[0] for r in results]
#     n = min(len(results), 3)
#     suggestions = names[:2]
#     return responses[n].format(*names), params, suggestions, excluded

# # Initialize the empty dictionary and lists
# params, suggestions, excluded = {}, [], []

# # Send the messages
# for message in ["I want a mid range hotel", "no that doesn't work for me"]:
#     print("USER: {}".format(message))
#     response, params, suggestions, excluded = respond(message, params, suggestions, excluded)
#     print("BOT: {}".format(response))

Respuestas:

    USER: I want a mid range hotel
    BOT: Hotel for Dogs is one option, but I know others too :)
    USER: no that doesn't work for me
    BOT: Grand Hotel is one option, but I know others too :)

## 3.4.6 Pending actions I

In [ ]:
# Define policy()
def policy(intent):
    # Return "do_pending" if the intent is "affirm"
    if intent == "affirm":
        return "do_pending", None
    # Return "Ok" if the intent is "deny"
    if intent == "deny":
        return "Ok", None
    if intent == "order":
        return "Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?", "Alright, I've ordered that for you!"

## 3.4.7 Pending actions II

In [ ]:
print('Ok_')

Ok_
